In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0223/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0223/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7ffaaa032c10>)

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
#DF 二維 有行列 
#series 一維 有自己apply

#替換 series.replace({"原本值":"改成"})
#有幾種不同的值 series.unique
#數出現次數   series.value_counts
u = train_df["作者"].unique()
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)}

y_train = train_df["作者"].replace(name2cat)
y_test = test_df["作者"].replace(name2cat)
name2cat

{'李白': 0, '杜甫': 1, '白居易': 2}

In [ ]:
#函式也是一種資料
#def test(): retun int  test()(5.3)

# Series.apply(func)
  #s = pd.Series([2, 3, 4])
  #def test(x):
   #return x * x
  #s.apply(test)

import jieba
def poemcut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

In [ ]:
#minmaxscale fit transfrom
#fit -> min:0 max:100
#transform -> p-min max-min:p
#測試資料不要再做一次fit
#! 訓練資料fit、transform
#! 測試資料transform (測試資料出現沒有在訓練資料中的資料:忽略)

from sklearn.feature_extraction.text import  CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

<30x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 352 stored elements in Compressed Sparse Row format>

In [ ]:
# sparse matrix (稀疏矩陣)
# fit_trasform 已經去掉標點符號
# 收集資料時有一點關係就放進去 
# naive bayes 單純貝式
vec.vocabulary_
x_train_count

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [ ]:
# alpha -> 保底次數
# 根據基底次數決定alpha 大約在 1/100 1/1000
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8333333333333334

In [ ]:
# confusion_martix 混淆矩陣
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + "(原本)" for name in u],
      columns=[name + "(預測)" for name in u])


In [ ]:
p = input("write:")
test = vec.transform([poemcut(p)])
prob = clf.predict_proba(test)[0]
for i in range(len(u)):
  print(u[i],round(prob[i], 3))